In [11]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [12]:
import pandas as pd
import logging
import uuid
from sqlmodel import Session, select
from core.db import engine
from models.vault_performance import VaultPerformance
from services import solv_service
from bg_tasks.utils import calculate_roi

logger = logging.getLogger(__name__)

def fix_solv_performance():
    """Fix and update vault performance data with correct APY calculations."""
    
    # Load historical performance data
    logger.info("Loading historical performance data...")
    df_performance = pd.read_csv(
        '../data/backup-performance-solv-1733715959243.csv',
        parse_dates=['datetime'],  # This will still attempt to parse dates
        dtype={'datetime': str}  # Read 'datetime' as string to avoid parsing issues
    )

    # Preprocess 'datetime' to standardize the format
    df_performance['datetime'] = df_performance['datetime'].str.replace(
        r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})(\.\d+)?(\+\d{2})', 
        r'\1\3', 
        regex=True
    )
    
    # Convert 'datetime' to datetime format, handling both formats
    df_performance['datetime'] = pd.to_datetime(df_performance['datetime'], errors='coerce', utc=True)

    # Resample to daily frequency
    df_performance = df_performance.set_index('datetime')
    df_performance = df_performance.resample('D').last().reset_index()
    
    # Fetch NAV data
    logger.info("Fetching NAV data from Solv...")
    df_nav = solv_service.fetch_nav_data()
    if df_nav is None:
        logger.error("Failed to fetch NAV data")
        return
    
    # Calculate corrected APYs for each day
    logger.info("Calculating corrected APYs...")
    
    with Session(engine) as session:
        for idx, row in df_performance.iterrows():
            current_date = pd.Timestamp(row['datetime']).tz_convert('UTC')  # Ensure current_date is UTC aware
            
            # Calculate 1-month APY
            one_month_ago = current_date - pd.DateOffset(months=1)
            
            # Ensure navDate is timezone-aware
            df_nav['navDate'] = pd.to_datetime(df_nav['navDate'], utc=True)  # Convert navDate to UTC if not already

            recent_nav = df_nav[df_nav['navDate'] <= current_date].iloc[-1]['adjustedNav']
            month_ago_nav = df_nav[df_nav['navDate'] <= one_month_ago].iloc[-1]['adjustedNav']
            days = (current_date - one_month_ago).days
            apy_1m = calculate_roi(recent_nav, month_ago_nav, days) * 100
            
            # Calculate 1-week APY
            one_week_ago = current_date - pd.DateOffset(weeks=1)
            week_ago_nav = df_nav[df_nav['navDate'] <= one_week_ago].iloc[-1]['adjustedNav']
            days = (current_date - one_week_ago).days
            apy_1w = calculate_roi(recent_nav, week_ago_nav, days) * 100
            
            # Insert new record
            new_performance = VaultPerformance(
                datetime=row['datetime'],
                total_locked_value=row['total_locked_value'] if pd.notna(row['total_locked_value']) else 0.0,  # Default to 0.0 if NaT
                benchmark=row['benchmark'] if pd.notna(row['benchmark']) else 0,  # Default to empty string if NaT
                pct_benchmark=row['pct_benchmark'] if pd.notna(row['pct_benchmark']) else 0.0,  # Default to 0.0 if NaT
                apy_1m=apy_1m,
                apy_1w=apy_1w,
                apy_ytd=row['apy_ytd'] if pd.notna(row['apy_ytd']) else 0.0,  # Default to 0.0 if NaT
                vault_id='2f782919-aa2f-403c-815d-37270c804660',  # Default to None if NaT
                risk_factor=row['risk_factor'] if pd.notna(row['risk_factor']) else 0.0,  # Default to 0.0 if NaT
                all_time_high_per_share=row['all_time_high_per_share'] if pd.notna(row['all_time_high_per_share']) else 0.0,  # Default to 0.0 if NaT
                total_shares=row['total_shares'] if pd.notna(row['total_shares']) else 0.0,  # Default to 0.0 if NaT
                sortino_ratio=row['sortino_ratio'] if pd.notna(row['sortino_ratio']) else 0.0,  # Default to 0.0 if NaT
                downside_risk=row['downside_risk'] if pd.notna(row['downside_risk']) else 0.0,  # Default to 0.0 if NaT
                unique_depositors=row['unique_depositors'] if pd.notna(row['unique_depositors']) else 0,  # Default to 0 if NaT
                earned_fee=row['earned_fee'] if pd.notna(row['earned_fee']) else 0.0,  # Default to 0.0 if NaT
                fee_structure=row['fee_structure'] if pd.notna(row['fee_structure']) else '',  # Default to empty string if NaT
                projected_apy=row['projected_apy'] if pd.notna(row['projected_apy']) else 0.0,  # Default to 0.0 if NaT
                id=uuid.uuid4(),  # Generate new UUID for each record
            )
            session.add(new_performance)
            session.commit()
            logger.info(f"Inserted performance for {row['datetime']}: 1M={apy_1m:.2f}%, 1W={apy_1w:.2f}%")


logging.basicConfig(level=logging.INFO)
fix_solv_performance()

INFO:__main__:Loading historical performance data...
INFO:__main__:Fetching NAV data from Solv...
INFO:__main__:Calculating corrected APYs...
INFO:__main__:Inserted performance for 2024-07-23 00:00:00+00:00: 1M=7.15%, 1W=25.55%
INFO:__main__:Inserted performance for 2024-07-24 00:00:00+00:00: 1M=7.69%, 1W=29.85%
INFO:__main__:Inserted performance for 2024-07-25 00:00:00+00:00: 1M=6.40%, 1W=29.85%
INFO:__main__:Inserted performance for 2024-07-26 00:00:00+00:00: 1M=7.33%, 1W=13.04%
INFO:__main__:Inserted performance for 2024-07-27 00:00:00+00:00: 1M=7.24%, 1W=13.04%
INFO:__main__:Inserted performance for 2024-07-28 00:00:00+00:00: 1M=7.92%, 1W=10.57%
INFO:__main__:Inserted performance for 2024-07-29 00:00:00+00:00: 1M=7.92%, 1W=10.57%
INFO:__main__:Inserted performance for 2024-07-30 00:00:00+00:00: 1M=8.75%, 1W=14.84%
INFO:__main__:Inserted performance for 2024-07-31 00:00:00+00:00: 1M=9.24%, 1W=14.63%
INFO:__main__:Inserted performance for 2024-08-01 00:00:00+00:00: 1M=8.89%, 1W=14.63